In [31]:
import json
import moment

datafile = 'data_april3_1am.json'
#datafile = 'data_july10.json'
datafile_base = datafile.replace('.json', '')
alldata = json.load(open(datafile))

In [47]:
def get_firstlast_info(experiment_info_with_sessions):
  first_timestamp = None
  last_timestamp = None
  first_localepoch = None
  last_localepoch = None
  for experiment_info in experiment_info_with_sessions:
    for condition_info in experiment_info['condition_info_list']:
      for day_info in condition_info['day_info_list']:
        for session_info in day_info['session_info_list']:
          timestamp = session_info['timestamp']
          localepoch = session_info['localepoch']
          if first_localepoch == None:
            first_localepoch = localepoch
          else:
            first_localepoch = min(first_localepoch, localepoch)
          if last_localepoch == None:
            last_localepoch = localepoch
          else:
            last_localepoch = max(last_localepoch, localepoch)
          if first_timestamp == None:
            first_timestamp = timestamp
          else:
            first_timestamp = min(first_timestamp, timestamp)
          if last_timestamp == None:
            last_timestamp = timestamp
          else:
            last_timestamp = max(last_timestamp, timestamp)
  return {
    'first_timestamp': first_timestamp,
    'last_timestamp': last_timestamp,
    'first_localepoch': first_localepoch,
    'last_localepoch': last_localepoch,
  }

def get_global_max_timestamp(alldata):
  max_timestamp = None
  for install_id,experiment_info_with_sessions in alldata.items():
    for experiment_info in experiment_info_with_sessions:
      for condition_info in experiment_info['condition_info_list']:
        for day_info in condition_info['day_info_list']:
          for session_info in sorted(day_info['session_info_list'], key=lambda k: k['timestamp']):
            timestamp = session_info['timestamp']
            if max_timestamp == None:
              max_timestamp = timestamp
            max_timestamp = max(max_timestamp, timestamp)
  return max_timestamp

def misc_analysis(alldata):
  max_timestamp = get_global_max_timestamp(alldata)
  attrition_num_days_same = 0
  total_num_days_same = 0
  attrition_num_days_random = 0
  total_num_days_random = 0
  attrition_num_days_switch = 0
  total_num_days_switch = 0
  attrition_num_days_nonswitch = 0
  total_num_days_nonswitch = 0
  attrition_num_days_switchtosame = 0
  total_num_days_switchtosame = 0
  attrition_num_days_switchtorandom = 0
  total_num_days_switchtorandom = 0
  for install_id,experiment_info_with_sessions in alldata.items():
    attrition_num_days_switch_user = 0
    total_num_days_switch_user = 0
    attrition_num_days_nonswitch_user = 0
    total_num_days_nonswitch_user = 0
    
    firstlast_info = get_firstlast_info(experiment_info_with_sessions)
    first_timestamp = firstlast_info['first_timestamp']
    last_timestamp = firstlast_info['last_timestamp']
    first_localepoch = firstlast_info['first_localepoch']
    last_localepoch = firstlast_info['last_localepoch']
    
    is_switch_day = False
    prev_day_condition = None
    for experiment_info in experiment_info_with_sessions:
      if len(experiment_info['condition_info_list']) != 2:
        print('wrong length')
      for idx,condition_info in enumerate(experiment_info['condition_info_list']):
        conditionduration = condition_info['conditionduration']
        if not (conditionduration > 1):
          continue
        for day_info in condition_info['day_info_list']:
          last_timestamp_on_day = None
          cur_day_condition = condition_info['condition']
          if prev_day_condition == None:
            is_switch_day = False
          elif prev_day_condition != cur_day_condition:
            is_switch_day = True
          else:
            is_switch_day = False
          prev_day_condition = cur_day_condition
          for session_info in sorted(day_info['session_info_list'], key=lambda k: k['timestamp']):
            time_spent = session_info['time_spent']
            timestamp = session_info['timestamp']
            if last_timestamp_on_day == None:
              last_timestamp_on_day = timestamp
            last_timestamp_on_day = max(last_timestamp_on_day, timestamp)
          is_last_day = int(last_timestamp_on_day == last_timestamp)
          attritioned_today = 0
          if len(day_info['session_info_list']) > 0 and is_last_day:
            if (moment.unix(max_timestamp) - moment.unix(last_timestamp)).days > 2:
              attritioned_today = 1
          if len(day_info['session_info_list']) > 0:
            if not is_switch_day:
              if cur_day_condition == 'random':
                total_num_days_random += 1
                if attritioned_today == 1:
                  attrition_num_days_random += 1
              else:
                total_num_days_same += 1
                if attritioned_today == 1:
                  attrition_num_days_same += 1
            if is_switch_day:
              total_num_days_switch += 1
              if attritioned_today == 1:
                attrition_num_days_switch += 1
              if cur_day_condition == 'same':
                total_num_days_switchtosame += 1
                if attritioned_today == 1:
                  attrition_num_days_switchtosame += 1
              else:
                total_num_days_switchtorandom += 1
                if attritioned_today == 1:
                  attrition_num_days_switchtorandom += 1
            else:
              total_num_days_nonswitch += 1
              if attritioned_today == 1:
                attrition_num_days_nonswitch += 1
          #if domain not in domain_to_last_timestamp:
          #  domain_to_last_timestamp[domain] = timestamp
          #else:
          #  domain_to_last_timestamp[domain] = max(timestamp, domain_to_last_timestamp[domain])
    #if total_num_days_nonswitch_user > 0 and total_num_days_switch_user > 0:
    #if True:
    #  attrition_num_days_switch += attrition_num_days_switch_user
    #  total_num_days_switch += total_num_days_switch_user
    #  attrition_num_days_nonswitch += attrition_num_days_nonswitch_user
    #  total_num_days_nonswitch += total_num_days_nonswitch_user
  return {
    'attrition_num_days_switch': attrition_num_days_switch,
    'total_num_days_switch': total_num_days_switch,
    'ratio_num_days_switch': attrition_num_days_switch / total_num_days_switch,
    'ratio_num_days_switchtosame': attrition_num_days_switchtosame / total_num_days_switchtosame,
    'ratio_num_days_switchtorandom': attrition_num_days_switchtorandom / total_num_days_switchtorandom,
    'attrition_num_days_nonswitch': attrition_num_days_nonswitch,
    'total_num_days_nonswitch': total_num_days_nonswitch,
    'ratio_num_days_nonswitch': attrition_num_days_nonswitch / total_num_days_nonswitch,
    'total_num_days_switchtosame': total_num_days_switchtosame,
    'attrition_num_days_switchtosame': attrition_num_days_switchtosame,
    'total_num_days_switchtorandom': total_num_days_switchtorandom,
    'attrition_num_days_switchtorandom': attrition_num_days_switchtorandom,
    'total_num_days_random': total_num_days_random,
    'total_num_days_same': total_num_days_same,
    'attritioned_num_days_random': attrition_num_days_random,
    'attritioned_num_days_same': attrition_num_days_same,
    'ratio_num_days_random': attrition_num_days_random / total_num_days_random,
    'ratio_num_days_same': attrition_num_days_same / total_num_days_same,
  }

results = misc_analysis(alldata)
print(results)

{'attrition_num_days_switch': 29, 'total_num_days_switch': 91, 'ratio_num_days_switch': 0.31868131868131866, 'ratio_num_days_switchtosame': 0.18181818181818182, 'ratio_num_days_switchtorandom': 0.36231884057971014, 'attrition_num_days_nonswitch': 90, 'total_num_days_nonswitch': 510, 'ratio_num_days_nonswitch': 0.17647058823529413, 'total_num_days_switchtosame': 22, 'attrition_num_days_switchtosame': 4, 'total_num_days_switchtorandom': 69, 'attrition_num_days_switchtorandom': 25, 'total_num_days_random': 311, 'total_num_days_same': 199, 'attritioned_num_days_random': 62, 'attritioned_num_days_same': 28, 'ratio_num_days_random': 0.19935691318327975, 'ratio_num_days_same': 0.1407035175879397}


In [39]:
import scipy.stats as stats
print(stats.fisher_exact([[29, 91 - 29], [90, 510 - 90]]))

(2.182795698924731, 0.002640985864413865)


In [40]:
print(stats.fisher_exact([[4, 22 - 4], [25, 69 - 25]]))

(0.39111111111111113, 0.18776704317133214)


In [43]:
print(stats.fisher_exact([[25, 69 - 25], [90, 510 - 90]]))

(2.6515151515151514, 0.0006364449939394982)
